In [99]:
import pandas as pd
ASX_data = pd.read_csv('https://www.asx.com.au/asx/research/ASXListedCompanies.csv',skiprows=1)
# format to align with yahoo finance ticker
ASX_data["Ticker"] = ASX_data['ASX code']+".AX"
temp = ASX_data[(ASX_data['GICS industry group'] == "Banks")]

#temp = ASX_data[(ASX_data['ASX code'] == "CBA") + (ASX_data['ASX code'] == "BEN") + (ASX_data['ASX code'] == "YBR")]
ASX_TICKER_LIST = list(temp['Ticker'])
ASX_TICKER_LIST


['ANZ.AX',
 'AFG.AX',
 'ABA.AX',
 'BOQ.AX',
 'BEN.AX',
 'BBC.AX',
 'CBA.AX',
 'GMA.AX',
 'HGH.AX',
 'KSL.AX',
 'MOC.AX',
 'MYS.AX',
 'N1H.AX',
 'NAB.AX',
 'RMC.AX',
 'VUK.AX',
 'WBC.AX',
 'YBR.AX']

In [100]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import seaborn as sns


import yfinance as yf
import yahoofinancials as yfin

In [ ]:
import time
start_time = time.time()


INF_DF = pd.DataFrame()
CALC_FEATURES_DF = pd.DataFrame()

def Info_Extract(TickName):
    
    global INF_DF
    global CALC_FEATURES_DF
    
    try:
        tick = yf.Ticker(TickName)
        
        INF = tick.info
        FIN = tick.financials.transpose()
        BS = tick.balance_sheet.transpose()
        DIV = tick.dividends        
        
        INF_DF = INF_DF.append(INF, ignore_index=True) 

   #----------------------------------------------------------------------------------------------------------
        #Part 0: Calculate WACC return
        
        # Calculate TAX rate
        try:
            TAX_RATE = (FIN['Income Tax Expense']/FIN['Income Before Tax']).median()
        except IndexError: 
            TAX_RATE = 99
    
        
        try:
            BS['CALC Capital'] = BS['Long Term Debt']+BS['Short Long Term Debt']+BS['Total Stockholder Equity']
            debt_p = (BS['Long Term Debt']+BS['Short Long Term Debt'])/BS['CALC Capital']
            DEBT_PERC_CAPITAL = debt_p.iloc[[0]].max()
            EQUITY_PERC_CAPITAL = 1-DEBT_PERC_CAPITAL
        except IndexError:
            debt_p = 99
            DEBT_PERC_CAPITAL = 99 
            EQUITY_PERC_CAPITAL = 0
        
        #calculate cost of debt
        try:
            debt_val = BS['Long Term Debt']/BS['Total Assets']
            DEBT_COST = debt_val.median()
        except Index_Error:
            DEBT_COST = 99

        
        #calculate equity rate using 2 different approach

        #Method 1 use the CAPM model
        rf_rtn = .025
        if INF["dividendYield"] is None:
            EQUITY_COST_CAPM = rf_rtn + ((-rf_rtn)*INF["beta"])
        else: 
            EQUITY_COST_CAPM = rf_rtn + ((INF["dividendYield"]-rf_rtn)*INF["beta"])
    
        
            
        #Method 2 use dividend approach
        div_growth = 0
        
        if INF["dividendRate"] is None:
            EQUITY_COST_DIV = 99
        else:
            EQUITY_COST_DIV = (INF["dividendRate"]/INF["previousClose"])+div_growth
            
        #Calculate minimum
        EQUITY_COST_AVG = min(EQUITY_COST_CAPM,EQUITY_COST_DIV)       

        WACC_CAPM =(EQUITY_PERC_CAPITAL*EQUITY_COST_CAPM)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)       
        
        WACC_DIV =(EQUITY_PERC_CAPITAL*EQUITY_COST_DIV)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)  
        
        WACC_AVG =(EQUITY_PERC_CAPITAL*EQUITY_COST_AVG)+(DEBT_PERC_CAPITAL*DEBT_COST)*(1-TAX_RATE)  
        
        
    #----------------------------------------------------------------------------------------------------------
         
        #Part 1: Dividend
        try:          
            DIV_PA = DIV.resample('Y').sum()
            DIV_L4_AVG = DIV_PA.iloc[[-5,-4,-3,-2]].sum()/4
            DIV_L4_STD = DIV_PA.iloc[[-5,-4,-3,-2]].std()
            DIVIDEND_BASE = DIV_L4_AVG - DIV_L4_STD   
        except IndexError:
            DIV_L4_AVG = 0
            DIV_L4_STD = 0
            DIVIDEND_BASE = 0    
            
        #Calculate NPV dividends
        
        DIVIDEND_BASE_NPV = (DIVIDEND_BASE/(1+WACC_AVG)**1) + (DIVIDEND_BASE/(1+WACC_AVG)**2) + (DIVIDEND_BASE/(1+WACC_AVG)**3)
        
        
        
      
    #----------------------------------------------------------------------------------------------------------
        
        YSTDAY_PRICE = INF['previousClose']
        
  
            
            
        #Part 2: Company Growth
        #Quarterly earling growth
        if INF['earningsQuarterlyGrowth'] is None:
            REV_GWTH_PA = 0
        else: 
            REV_GWTH_PA = INF['earningsQuarterlyGrowth']*4/100   
                

            
        #Part 4: Valuation
        #Net tangible assets
        try:
            VAL_NTA = BS['Net Tangible Assets'].iloc[[0]].max()/INF['floatShares']
        except IndexError:
            VAL_NTA = 0
        
        #PE ratio
        try:
            VAL_PE_RATIO = (INF['enterpriseValue']/INF['enterpriseToRevenue']) * INF['profitMargins']/INF['sharesOutstanding']*float(INF['trailingPE'])
        except IndexError:
            VAL_PE_RATIO = 0
        
        REV_YOY = FIN['Total Revenue'].iloc[[0]].max()/ FIN['Total Revenue'].iloc[[1]].max() -1
        REVENUE_AVG=FIN['Total Revenue'].mean()
        TAX_AVG =FIN['Income Tax Expense'].mean()
        OPEX_AVG=FIN['Total Operating Expenses'].mean()
        NET_INCOME = REVENUE_AVG - TAX_AVG - OPEX_AVG
        OPEX_REV_RATE = OPEX_AVG / REVENUE_AVG        
            
            
 #----------------------------------------------------------------------------------------------------------          
       #NPV Cashflows 
        FIN_OG = tick.financials
        NET_INCOME_BASE = FIN_OG[(FIN_OG.index == "Net Income Applicable To Common Shares")].median().mean()
        NPV_NET_INCOME = (NET_INCOME_BASE/(1+WACC_AVG)**1) + (NET_INCOME_BASE/(1+WACC_AVG)**2) + (NET_INCOME_BASE/(1+WACC_AVG)**3)
        
        NPV_NET_INCOME_PERP = NET_INCOME_BASE / WACC_AVG /INF['sharesOutstanding'] 
        NPV_DIVIDEND_BASE_PERP = DIVIDEND_BASE / WACC_AVG
      
        POSSIBLE_GAIN_REV = NPV_NET_INCOME_PERP - YSTDAY_PRICE
    
        #Append features
        
        x_list = [TickName,TAX_RATE,YSTDAY_PRICE,DIV_L4_AVG,DIV_L4_STD,DIVIDEND_BASE,REV_GWTH_PA,VAL_NTA,VAL_PE_RATIO,
                 DEBT_PERC_CAPITAL,EQUITY_PERC_CAPITAL,DEBT_COST,
                  EQUITY_COST_CAPM,EQUITY_COST_DIV,EQUITY_COST_AVG,WACC_CAPM,WACC_DIV,WACC_AVG,DIVIDEND_BASE_NPV,
                  REV_YOY,REVENUE_AVG,TAX_AVG,OPEX_AVG,NET_INCOME,OPEX_REV_RATE,
                  NET_INCOME_BASE,NPV_NET_INCOME,NPV_NET_INCOME_PERP,NPV_DIVIDEND_BASE_PERP,POSSIBLE_GAIN_REV
                 ]
        #,DEBT_COST,EQUITY_COST_CAPM,EQUITY_COST_DIV,EQUITY_COST_AVG]
        x_list_df = pd.DataFrame(x_list).transpose()
        CALC_FEATURES_DF = CALC_FEATURES_DF.append(x_list_df, ignore_index=True)          
                

        
        
    except Exception:
        print("Error with",TickName)
        pass

    return()


for company in ASX_TICKER_LIST:
    Info_Extract(company)



print("--- %s seconds ---" % (time.time() - start_time))



Error with BOQ.AX
Error with BBC.AX
Error with GMA.AX
Error with HGH.AX
Error with KSL.AX
Error with MOC.AX


In [98]:
CALC_FEATURES_DF.columns = ['TickName','TAX_RATE','YSTDAY_PRICE','DIV_L4_AVG','DIV_L4_STD','DIVIDEND_BASE',
                            'REV_GWTH_PA','VAL_NTA','VAL_PE_RATIO','DEBT_PERC_CAPITAL','EQUITY_PERC_CAPITAL',
                            'DEBT_COST','EQUITY_COST_CAPM','EQUITY_COST_DIV','EQUITY_COST_AVG',
                            'WACC_CAPM','WACC_DIV','WACC_AVG','DIVIDEND_BASE_NPV',
                            'REV_YOY','REVENUE_AVG','TAX_AVG','OPEX_AVG','NET_INCOME','OPEX_REV_RATE',
                            'NET_INCOME_BASE','NPV_NET_INCOME','NPV_NET_INCOME_PERP','NPV_DIVIDEND_BASE_PERP',
                            'POSSIBLE_GAIN_REV'
                           ]
CALC_FEATURES_DF


,TickName,TAX_RATE,YSTDAY_PRICE,DIV_L4_AVG,DIV_L4_STD,DIVIDEND_BASE,REV_GWTH_PA,VAL_NTA,VAL_PE_RATIO,DEBT_PERC_CAPITAL,...,REVENUE_AVG,TAX_AVG,OPEX_AVG,NET_INCOME,OPEX_REV_RATE,NET_INCOME_BASE,NPV_NET_INCOME,NPV_NET_INCOME_PERP,NPV_DIVIDEND_BASE_PERP,POSSIBLE_GAIN_REV
0,BEN.AX,0.31573,6.99,0.69,0.011547,0.678453,-0.01128,8.09416,7.87357,0.471896,...,1.48528e+09,1.913e+08,9.24475e+08,3.695e+08,0.622427,4.14125e+08,1.09189e+09,12.4587,10.0593,5.46868
1,CBA.AX,0.286877,66.3,4.2775,0.0525198,4.22498,0.0136,34.9201,71.7227,0.728974,...,2.31058e+10,3.63175e+09,9.73425e+09,9.73975e+09,0.421291,9.25025e+09,2.31081e+10,53.6269,43.3597,-12.6731
2,YBR.AX,0.195457,0.07,0,0,0,0,0.23125,0.0522173,0.168059,...,2.14858e+08,-1.988e+06,2.04876e+08,1.197e+07,0.953541,-1.16362e+07,-3.48225e+07,-28.9766,0,-29.0466


In [86]:
tick = yf.Ticker("CBA.AX")
        
INF = tick.info
FIN = tick.financials
BS = tick.balance_sheet.transpose()
DIV = tick.dividends    

In [94]:
FIN_OG = tick.financials
temp = FIN_OG[(FIN_OG.index == "Net Income Applicable To Common Shares")].median().mean()

temp

9250250000.0

In [29]:
INF_DF.to_excel("/Users/JZ/Downloads/LATEST_TEST.xlsx", sheet_name='INFO_BANKS')
CALC_FEATURES_DF.to_excel("/Users/JZ/Downloads/LATEST_TEST_FEATURES.xlsx", sheet_name='CREATED_FEATURES')